## Problema 4: Ruta de Mínimo Costo en una Red de Nodos Móviles Inalámbricos
## 1. Conjuntos

- $N$: Conjunto de todos los nodos en la red.  
  En este caso, $N = \{1, 2, 3, 4, 5, 6, 7\}$.  
  - Nodo 4: nodo de **origen**  
  - Nodo 6: nodo de **destino**  

## 2. Parámetros

- $Cost_{ij}$: Costo de viajar del nodo $i$ al nodo $j$.  
  Este valor se obtiene a partir de la **distancia euclidiana** (si esta es $\leq 20$) entre los nodos $i$ y $j$.  
  - Si la distancia $\leq 20$, entonces $cost_{i,j} = \text{distancia euclidiana}(i, j)$.  
  - Si la distancia $> 20$, se establece un valor alto (p.e. 999) que representa la imposibilidad de viajar directamente entre esos nodos.

## 3. Variables de decisión

- $X_{ij}$: Variable binaria  
- Si se usa el arco que va del nodo i al nodo j, entonces $X_{ij}$= 1
- De lo contrario, $X_{ij}$ = 0

## 4. Función objetivo

Buscamos **minimizar** el **costo total** de la ruta en la red:
 $min \sum_{i \in N} \sum_{j \in N} Cost_{ij}X_{ij}$


## 5. Restricciones

1. **Flujo que sale del nodo 4** (origen):  
   Solo un arco debe salir del nodo 4, asegurando que la ruta inicia desde allí:
   - $\sum_{j \in N} X_{4j} = 1$  


2. **Flujo que entra al nodo 6** (destino):  
   Solo un arco debe entrar al nodo 6, para que la ruta termine allí:
   - $\sum_{i \in N} X_{i6} = 1$  


3. **El flujo que entra es el mismo que sale** en nodos intermedios ($i \neq 4, i \neq 6$):  

- $\sum_{j \in N} X_{ij} - \sum_{j \in N} X_{ji} = 0  \quad \forall i \in N | i \neq 4 \wedge  i \neq 6 $  


## 6. Tipo de problema

Este es un MIP, dado que:
- Las variables de decisión $x_{i,j}$ son **binarias**.
- La **función objetivo** es lineal (suma de productos entre costos y variables).
- Las **restricciones** son también lineales.


### preprocesamiento datos:


 La función de preprocesamiento de datos se realizó a través de la creación de una matriz que representa los costos de viajar entre cada par de nodos. Esta matriz se construyó utilizando las coordenadas (x, y) de cada nodo para calcular la distancia euclidiana entre ellos. Si la distancia euclidiana es menor o igual a 20, se considera como el costo de viajar entre los nodos. De lo contrario, se establece un costo alto (999) para indicar que no es posible viajar directamente entre esos nodos.

 La ecuación utilizada para calcular la distancia euclidiana fue:
 $d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$


In [28]:
def calcular_distancia_euclidiana(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def crearMatriz():
    PosicionesNodos = {
    1: {"CoordX": 20, "CoordY": 6}, # Nodo 1
    2: {"CoordX": 22, "CoordY": 1}, # Nodo 2
    3: {"CoordX": 9, "CoordY": 2}, # Nodo 3
    4: {"CoordX": 3, "CoordY": 25}, # Nodo 4
    5: {"CoordX": 21, "CoordY": 10}, # Nodo 5
    6: {"CoordX": 29, "CoordY": 2}, # Nodo 6
    7: {"CoordX": 14, "CoordY": 12}, # Nodo 7
}
    matriz = {}
    for i in PosicionesNodos:
        for j in PosicionesNodos:
            if i == j:
                matriz[i, j] = 999
            else:
                distancia_euclidiana = calcular_distancia_euclidiana(
                    PosicionesNodos[i]["CoordX"], PosicionesNodos[i]["CoordY"], 
                    PosicionesNodos[j]["CoordX"], PosicionesNodos[j]["CoordY"])
                matriz[i, j] = distancia_euclidiana if distancia_euclidiana <= 20 else 999
    return matriz

print(crearMatriz())


{(1, 1): 999, (1, 2): 5.385164807134504, (1, 3): 11.704699910719626, (1, 4): 999, (1, 5): 4.123105625617661, (1, 6): 9.848857801796104, (1, 7): 8.48528137423857, (2, 1): 5.385164807134504, (2, 2): 999, (2, 3): 13.038404810405298, (2, 4): 999, (2, 5): 9.055385138137417, (2, 6): 7.0710678118654755, (2, 7): 13.601470508735444, (3, 1): 11.704699910719626, (3, 2): 13.038404810405298, (3, 3): 999, (3, 4): 999, (3, 5): 14.422205101855956, (3, 6): 20.0, (3, 7): 11.180339887498949, (4, 1): 999, (4, 2): 999, (4, 3): 999, (4, 4): 999, (4, 5): 999, (4, 6): 999, (4, 7): 17.029386365926403, (5, 1): 4.123105625617661, (5, 2): 9.055385138137417, (5, 3): 14.422205101855956, (5, 4): 999, (5, 5): 999, (5, 6): 11.313708498984761, (5, 7): 7.280109889280518, (6, 1): 9.848857801796104, (6, 2): 7.0710678118654755, (6, 3): 20.0, (6, 4): 999, (6, 5): 11.313708498984761, (6, 6): 999, (6, 7): 18.027756377319946, (7, 1): 8.48528137423857, (7, 2): 13.601470508735444, (7, 3): 11.180339887498949, (7, 4): 17.029386365

### Implementación de la formulación en pyomo:

In [29]:

from __future__ import division
from pyomo.environ import *

from pyomo.opt import SolverFactory

import sys
import os

os.system("clear")

#modelo
Model = ConcreteModel()

numNodes=7

N=RangeSet(1, numNodes)

# matriz de costos a partir de las distancias euclidianas
cost= crearMatriz()

# variable de decisión
Model.x = Var(N,N, domain=Binary)

# función objetivo
Model.obj = Objective(expr = sum(Model.x[i,j]*cost[i,j] for i in N for j in N), sense=minimize)

# restricciones
Model.res1=ConstraintList()
for i in N:
    if i==4:
        Model.res1.add(sum(Model.x[i,j] for j in N)==1)

Model.res2=ConstraintList()
for j in N:
    if j==6:
        Model.res2.add(sum(Model.x[i,j] for i in N)==1)

Model.res3=ConstraintList()
for i in N:
    if i!=4 and i!=6:
        Model.res3.add(sum(Model.x[i,j] for j in N) - sum(Model.x[j,i] for j in N)==0)
    
SolverFactory('glpk').solve(Model)

Model.display()





Model unknown

  Variables:
    x : Size=49, Index=[1:7]*[1:7]
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 3) :     0 :   0.0 :     1 : False : False : Binary
        (1, 4) :     0 :   0.0 :     1 : False : False : Binary
        (1, 5) :     0 :   0.0 :     1 : False : False : Binary
        (1, 6) :     0 :   0.0 :     1 : False : False : Binary
        (1, 7) :     0 :   0.0 :     1 : False : False : Binary
        (2, 1) :     0 :   0.0 :     1 : False : False : Binary
        (2, 2) :     0 :   0.0 :     1 : False : False : Binary
        (2, 3) :     0 :   0.0 :     1 : False : False : Binary
        (2, 4) :     0 :   0.0 :     1 : False : False : Binary
        (2, 5) :     0 :   0.0 :     1 : False : False : Binary
        (2, 6) :     0 :   0.0 :     1 : False : False : Binary
        (2, 7) :     0 :   0.0 :     1 : 

In [31]:
for i in N:
    for j in N:
        if Model.x[i, j].value == 1:
            print(f"Camino usado: {i} -> {j}")


Camino usado: 4 -> 7
Camino usado: 7 -> 6


## Analisis de resultados
 La ruta óptima entre los nodos 4 y 6 sigue el trayecto 4 → 7 → 6, con un costo total de 35.057 unidades. Este resultado es consistente con las restricciones del preprocesamiento de datos, donde las conexiones entre nodos solo son posibles si la distancia Euclidiana entre ellos es menor o igual a 20 unidades. El preprocesamiento es fundamental, ya que el calculo de las distancias era necesario para poder estructurar la matriz de costos y tener en cuenta solamente los validos.